### This notebook will help you to create the s3 buckets for output data generated by etl.py

* To access you aws account, you need to create a programmatic access <a href="https://docs.aws.amazon.com/IAM/latest/UserGuide/id_users_create.html">Reference Link</a>
* After create the access, pass the values of AWS_ACCESS_KEY_ID and AWS_SECRET_ACCESS_KEY inside dl.cfg

In [1]:
import configparser
from datetime import datetime
import os
import logging
import boto3
from botocore.exceptions import ClientError

config = configparser.ConfigParser()
config.read('dl.cfg')

os.environ['AWS_ACCESS_KEY_ID']=config['S3']['AWS_ACCESS_KEY_ID']
os.environ['AWS_SECRET_ACCESS_KEY']=config['S3']['AWS_SECRET_ACCESS_KEY']

In [4]:
def create_bucket(bucket_name, region=None):
    """Create an S3 bucket in a specified region

    If a region is not specified, the bucket is created in the S3 default
    region (us-east-1).

    :param bucket_name: Bucket to create
    :param region: String region to create bucket in, e.g., 'us-west-2'
    :return: True if bucket created, else False
    """

    # Create bucket
    try:
        if region is None:
            s3_client = boto3.client('s3')
            s3_client.create_bucket(Bucket=bucket_name)
        else:
            s3_client = boto3.client('s3', region_name=region)
            location = {'LocationConstraint': region}
            s3_client.create_bucket(Bucket=bucket_name,
                                    CreateBucketConfiguration=location)
    except ClientError as e:
        logging.error(e)
        return False
    return True

In [8]:
bucket_name = '' #WRITE THE OUTPUT BUCKET NAME HERE
config['S3']['OUTPUT_BUCKET'] = bucket_name
create_bucket(bucket_name)

#UPDATE dl.cfg
with open('dl.cfg','w') as conf:
    config.write(conf)
    
config.read('dl.cfg')

True

In [11]:
s3 = boto3.client('s3')

bucket_name = config['S3']['OUTPUT_BUCKET']

folders = ['artists','users','songs','times','songplays']

for folder in folders:
    s3.put_object(Bucket=bucket_name, Key=(folder+'/'))
    print(folder + ' created.')

artists created.
users created.
songs created.
times created.
songplays created.


### Now you can execute etl.py using the created folders to output directory.